In [1]:
!python ../setup.py build 
!python ../setup.py build_ext --inplace

First: /Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot
/Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot
running build
running build_py
file /Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot/opttrot.py (for module opttrot) not found
copying /Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot/graph_algs/dwave.py -> build/lib.macosx-11.1-arm64-cpython-312/graph_algs
copying /Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot/graph_algs/graph_opts.py -> build/lib.macosx-11.1-arm64-cpython-312/graph_algs
copying /Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot/graph_algs/__init__.py -> build/lib.macosx-11.1-arm64-cpython-312/graph_algs
file /Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot/opttrot.py (for module opttrot) not found
running build_ext
building 'c_utils' extension
clang -fno-strict-overflow -DNDEBUG -O2 -Wall -fPIC -O2 -isystem /Users/hyunseongkim/miniconda3/envs/opttrot/include -ar

In [2]:
import os, sys
sys.path.append("..")

## Vectorized commute

In [21]:
import opttrot
from opttrot.pauli import PauliElement, PauliPoly
from opttrot.pauli import get_commutes, get_commutes_sparse

In [22]:
arr = np.array([2**64+1])
arr

array([18446744073709551617], dtype=object)

In [23]:
a = PauliElement(0, 1, 2, 1.)
b = PauliElement(2, 1, 2, 1.)
c = PauliElement(3, 0, 2, 1)
d = PauliElement(0,0,2,1)
a, b, c

(PauliElement(n=2, weight=1.000000+(0.000000)j, IZ),
 PauliElement(n=2, weight=1.000000+(0.000000)j, XZ),
 PauliElement(n=2, weight=1.000000+(0.000000)j, XX))

In [24]:
ppoly = PauliPoly.from_iterables((a,b,c, d))

In [25]:
ppoly.poly

array([PauliElement(n=2, weight=1.000000+(0.000000)j, II),
       PauliElement(n=2, weight=1.000000+(0.000000)j, XX),
       PauliElement(n=2, weight=1.000000+(0.000000)j, IZ),
       PauliElement(n=2, weight=1.000000+(0.000000)j, XZ)], dtype=object)

In [26]:
a.commute(a), a.commute(b), a.commute(c)

(True, True, False)

In [27]:
get_commutes(ppoly.poly, a)

array([ True, False,  True,  True])

In [28]:
from scipy.sparse import coo_matrix
import numpy as np

In [29]:
coo = coo_matrix(ppoly.coef_matrix)

In [30]:
%%timeit
a.commute(b)

216 ns ± 2.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [31]:
get_commutes_sparse(coo.row.astype(np.uint64), coo.col.astype(np.uint64), a)

array([ True, False,  True,  True])

In [32]:
nx_s = np.bitwise_xor(coo.row, coo.col)
nz_s = coo.col

In [33]:
nx, nz = a.sym_code

In [35]:
from opttrot.utils import np_bitwise_count

In [36]:
commute_arr = np.bitwise_xor(np.bitwise_and(nx_s, nz), np.bitwise_and(nz_s, nx))
if commute_arr.max() !=1:
    commute_arr = np_bitwise_count(commute_arr)

version is 1.26
Parsing


array([0, 1, 0, 0], dtype=uint64)

In [37]:
np.bitwise_xor(np.array([1,1,1]), 1)

array([0, 0, 0])

In [45]:
%%timeit
get_commutes(ppoly.poly, a)

1.01 μs ± 9.37 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [47]:
%%timeit
get_commutes_sparse(coo.row.astype(np.uint64), coo.col.astype(np.uint64), a)

1.63 μs ± 8.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [46]:
%%timeit
p = ppoly.poly
n = np.zeros(len(p))
for i in range(len(p)):
    n[i] = a.commute(p[i])

1.3 μs ± 8.29 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# Optimization of Pauli Poly

There are two measure to estimate performances of `PauliPoly`.

- Naive `PauliPoly` implementation.
- Qiskit `SparsePauliOp` and `PauliList`.
- 
The PauliPoly in the origial OptTrot library was a naive implementation written by python.
In here, we will optimize the routine as soon as possible using scipy and numba routines.

In [3]:
import opttrot

In [4]:
from opttrot.pauli_c import get_paulilist_from_coefs

In [5]:
from opttrot.pauli_c import PauliElement
from opttrot.pauli_c import get_commutes

In [6]:
from opttrot.pauli import PauliElement
from opttrot.pauli import PauliPoly

import numpy as np
import matplotlib.pyplot as plt

In [11]:
from importlib import reload
#import paulipoly
#reload(paulipoly)
#from paulipoly import PauliPoly as NewPauliPoly
#import ten_con
#reload(ten_con)
#from ten_con import _mat_to_coef_mat, _coef_to_mat

In [12]:
from scipy.sparse import coo_matrix, csr_matrix

In [13]:
from qiskit.quantum_info import Pauli, PauliList, SparsePauliOp

In [14]:
from copy import deepcopy

In [15]:
qubits = 4
def get_test_hermit(qubits, tol= 0.5):
    ndim = int(2**qubits)
    A = np.random.rand(ndim, ndim)
    A[A<tol] = 0.
    H = A + 1j*A
    H = H.conj().T@H
    return H


In [11]:
A = np.sqrt(get_test_hermit(qubits, 0.96))
B = np.sqrt(get_test_hermit(qubits, 0.8))


In [12]:
test_ppoly = PauliPoly.from_matrix(A)

In [13]:
test_ppoly.terms

[((0.46814521442164375-4.699665136611769e-19j), 'IIII'),
 ((0.29591769496285136+5.422126478591218e-19j), 'IIIY'),
 ((-0.05277454759431581+3.6435990233084064e-19j), 'IIYI'),
 ((-0.22500206705310816+1.3765390638511395e-18j), 'IIYY'),
 ((-0.12260744408633287-2.7991520058422565e-19j), 'IYII'),
 ((0.04962007537245949-1.2920943621045244e-18j), 'IYIY'),
 ((-0.2927632227409951+3.85521811914562e-19j), 'IYYI'),
 ((-0.12053570328220273-6.266573496057367e-19j), 'IYYY'),
 ((0.11811312558204645-3.2165542045477387e-19j), 'YIII'),
 ((-0.05411439387674591+6.905237410655249e-19j), 'YIIY'),
 ((0.29725754124528153+2.1604880912443751e-19j), 'YIYI'),
 ((0.12503002178648917+1.2282279706447362e-18j), 'YIYY'),
 ((-0.12187554956463287-3.004935108710524e-19j), 'YYII'),
 ((0.05035196989415949-1.3126726723913512e-18j), 'YYIY'),
 ((-0.2934951172626951+4.0610012220138877e-19j), 'YYYI'),
 ((-0.12126759780390275-6.0607903931891e-19j), 'YYYY')]

In [14]:
#plist = list(test_ppoly._terms.values())
coef_mat = test_ppoly.coef_matrix
mat = test_ppoly.matrix
coef_mat[coef_mat<0.05] = 0.
cmat = csr_matrix(coef_mat)
100*cmat.data.size/(4**qubits)

2.34375

In [15]:
from opttrot.pauli import PauliElement

In [90]:
a = PauliElement(3, 4, 5, 0.5)
b = PauliElement(1, 2, 5, 0.5)
c = PauliElement(2, 2, 5, 0.1)

In [91]:
parr = np.array([a,b, c])
print(parr)

[PauliElement(n=5, weight=0.500000+(0.000000)j, IIZXX)
 PauliElement(n=5, weight=0.500000+(0.000000)j, IIIZX)
 PauliElement(n=5, weight=0.100000+(0.000000)j, IIIYI)]


In [92]:
ppoly = PauliPoly.from_iterables(parr)

In [93]:
ppoly.poly

array([PauliElement(n=5, weight=0.100000+(0.000000)j, IIIXI),
       PauliElement(n=5, weight=0.500000+(0.000000)j, IIIYZ),
       PauliElement(n=5, weight=0.500000+(0.000000)j, IIYZZ)],
      dtype=object)

In [79]:
a.commute(b)

False

In [80]:
a+b

PauliPoly(terms:2)[
((0.5+0j), 'IIIYZ'),
((0.5+0j), 'IIYZZ')
]

In [48]:
c.commute(a), c.commute(b)

(False, True)

In [49]:
c.commute(ppoly)

array([False,  True])

## Coef matrix to PauliPoly

In [15]:
%%timeit
PauliPoly(coef_mat)

2.9 μs ± 122 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Hermit matrix to PauliPoly

In [16]:
%%timeit
PauliPoly.from_matrix(mat)

26.8 μs ± 385 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [17]:
%%timeit
SparsePauliOp.from_operator(mat)

13.7 μs ± 278 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Test

## PauliPoly to Hermit

In [18]:
# PauliPoly
pp001 = PauliPoly.from_matrix(A)
pp002 = PauliPoly.from_matrix(B)

# Qiskit
spa001 = SparsePauliOp.from_operator(A)
spa002 = SparsePauliOp.from_operator(B)

In [19]:
%%timeit
pp001.matrix

15.4 μs ± 391 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [20]:
%%timeit
spa001.to_matrix() 

23.6 μs ± 2.21 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Tset: Qubit 12 (Matrix construction)

- PauliPoly: 3.48 s ± 12.3 ms
- Qiskit: 33.2 s ± 1.98 s

In [25]:
%%timeit
pp002.matrix

44 µs ± 587 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [26]:
%%timeit
spa002.to_matrix()

96.8 µs ± 561 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## PauliElement generation

In [21]:
pp001.matrix

matrix([[ 0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j],
        [ 0.00000000e+00+0.00000000e+00j,
          1.39188187e+00+6.44026808e-19j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          1.40240774e+00-1.95582036e-17j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.000000

In [22]:
ps = pp001.poly

In [23]:
ps

array([PauliElement(n=4, weight=nan+(nan)j, IIII),
       PauliElement(n=4, weight=nan+(nan)j, IZII),
       PauliElement(n=4, weight=nan+(nan)j, IZIZ),
       PauliElement(n=4, weight=nan+(nan)j, IZZZ),
       PauliElement(n=4, weight=nan+(nan)j, ZIIZ),
       PauliElement(n=4, weight=nan+(nan)j, ZZIZ),
       PauliElement(n=4, weight=nan+(nan)j, IIIY),
       PauliElement(n=4, weight=nan+(nan)j, IZIX),
       PauliElement(n=4, weight=nan+(nan)j, IZIY),
       PauliElement(n=4, weight=nan+(nan)j, IZZX),
       PauliElement(n=4, weight=nan+(nan)j, ZIIX),
       PauliElement(n=4, weight=nan+(nan)j, ZZIX),
       PauliElement(n=4, weight=nan+(nan)j, IIYI),
       PauliElement(n=4, weight=nan+(nan)j, IZXZ),
       PauliElement(n=4, weight=nan+(nan)j, IZYI),
       PauliElement(n=4, weight=nan+(nan)j, IZYZ),
       PauliElement(n=4, weight=nan+(nan)j, ZIYZ),
       PauliElement(n=4, weight=nan+(nan)j, ZZYZ),
       PauliElement(n=4, weight=nan+(nan)j, IIYY),
       PauliElement(n=4, weight

In [24]:
pp001.coef_matrix

matrix([[nan+nanj,  0. +0.j,  0. +0.j,  0. +0.j, nan+nanj, nan+nanj,
          0. +0.j, nan+nanj,  0. +0.j, nan+nanj,  0. +0.j,  0. +0.j,
          0. +0.j, nan+nanj,  0. +0.j,  0. +0.j],
        [ 0. +0.j, nan+nanj,  0. +0.j,  0. +0.j, nan+nanj, nan+nanj,
         nan+nanj,  0. +0.j, nan+nanj,  0. +0.j,  0. +0.j,  0. +0.j,
         nan+nanj,  0. +0.j,  0. +0.j,  0. +0.j],
        [ 0. +0.j,  0. +0.j, nan+nanj,  0. +0.j,  0. +0.j, nan+nanj,
         nan+nanj, nan+nanj,  0. +0.j,  0. +0.j,  0. +0.j, nan+nanj,
          0. +0.j,  0. +0.j,  0. +0.j, nan+nanj],
        [ 0. +0.j,  0. +0.j,  0. +0.j, nan+nanj, nan+nanj,  0. +0.j,
         nan+nanj, nan+nanj,  0. +0.j,  0. +0.j, nan+nanj,  0. +0.j,
          0. +0.j,  0. +0.j, nan+nanj,  0. +0.j],
        [nan+nanj, nan+nanj,  0. +0.j, nan+nanj, nan+nanj,  0. +0.j,
          0. +0.j,  0. +0.j,  0. +0.j, nan+nanj,  0. +0.j,  0. +0.j,
          0. +0.j, nan+nanj,  0. +0.j,  0. +0.j],
        [nan+nanj, nan+nanj, nan+nanj,  0. +0.j,  0. +0.j, n

## Addition

**Comparsion: Matrix and set methods**

Qubits = 5

- Matrix: 58.3 ms ± 2.4 ms
- Set: 13.2 ms ± 111 µs

Qubits = 8

- Matrix: 3.68 s ± 67 ms per loop
- Set: 2 s ± 74.5 ms

In [30]:
%%timeit
pp001+pp002

18.3 µs ± 279 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [31]:
%%timeit
spa001+spa002

40 µs ± 422 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Mat multiplication

In [12]:
%%timeit
pp001@pp002

NameError: name 'pp001' is not defined

In [36]:
%%timeit
spa001@spa002 # Unable to multiplicate.

138 ms ± 5.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Summary

A(xN.N): B took N.N times more slower than A.

|Opeartion|Faster|
|:-------:|:----:|
|o->mat   | Qiskit(x1.2) | 
|mat->o   | PauliPoly(x2-1.2) | 
|Addition | PauliPoly(x2) |
|Mat Mul  | PauliPoly(x Exponential) |
